Goal: pick 4 USA men's Gymnasts. Gymnasts cannot compete on the same appartaus more than once. And each apparatus needs to be competed on 3 times each, then get the largest sum of the top 3 out of 4 scores on each apparats






# Tabu Search

### IAA

Selected Athletes:
['khoi young' 'asher hong' 'fred richard' 'yul moldauer']
Total Score:
260.57900000000006

### Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
Total Score:
253.19598690476192

# Hyper-Heuristics:

### IAA
Best Selection: ['joshua karnes' 'ian lasic-ellis' 'asher hong' 'yul moldauer']
Best Score: 254.94

### Apparatus
Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191


#  Variable Neighborhood Search (VNS):

### IAA

Selected Athletes:
['yul moldauer' 'colt walker' 'khoi young' 'curran phillips']
Total Score:
176.531

### Apparatus
Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825


# brute-force search

### Apparatus
Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365

### IAA

Selected Athletes: ['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer'] Maximum Score: 173.974

# MILP

### Apparatus
not returning 4 athletes

### IAA
not returning 4 athletes

In [37]:

import pandas as pd
import numpy as np
from itertools import combinations
from pyqubo import Array, Constraint, Placeholder, solve_qubo
from pulp import LpProblem, LpVariable, lpSum, LpMaximize



In [38]:
df = pd.read_csv('../../Data/cleandata22-23/usa_men22_23.csv')
df1 = pd.read_csv('../../Cleaning/USOPC/usa_men_apparatus.csv')
df2=pd.read_csv('update_usa_apparatus.csv')
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 

In [39]:
usa = iaa24[iaa24['Nation'] == 'USA']


In [40]:
df=usa.copy()

In [44]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/3263395605.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


#  Variable Neighborhood Search (VNS):

## IAA

Selected Athletes:
['yul moldauer' 'colt walker' 'khoi young' 'curran phillips']
Total Score:
176.531

## Apparatus
Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825



---

## IAA

In [49]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/318956366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


In [51]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['yul moldauer' 'colt walker' 'khoi young' 'curran phillips']
Total Score:
176.531


## Apparatus
Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825



In [24]:
df = df2.copy()

In [26]:
df.sort_values(by='name', inplace=True)


In [12]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['name'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['name'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['name'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['name'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['colt walker' 'paul juda' 'khoi young' 'frederick nathaniel richard']
Total Score:
170.307825


# Hyper-Heuristics:

## IAA
Best Selection: ['joshua karnes' 'ian lasic-ellis' 'asher hong' 'yul moldauer']
Best Score: 254.94


## Apparatus
Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191

---

## IAA

In [52]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/318956366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


In [53]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['joshua karnes' 'ian lasic-ellis' 'asher hong' 'yul moldauer']
Best Score: 254.94


## Apparatus
Best Selection: ['khoi young' 'paul juda' 'brody malone' 'fuzzy benas']
Best Score: 250.58868690476191

In [10]:
df = df2.copy()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    58 non-null     object 
 1   fx      58 non-null     float64
 2   hb      58 non-null     float64
 3   pb      58 non-null     float64
 4   ph      58 non-null     float64
 5   sr      58 non-null     float64
 6   vt      58 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.3+ KB


In [13]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
# df = ...

# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['name'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['name'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['brody malone' 'paul juda' 'zachary nunez' 'khoi young']
Best Score: 250.58868690476191


# Tabu Search

## IAA

Selected Athletes:
['khoi young' 'asher hong' 'fred richard' 'yul moldauer']
Total Score:
260.57900000000006

## Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
Total Score:
253.19598690476192

---

## IAA

In [55]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])
df.head()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/1385056510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [56]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 246.274
Iteration 2, Best Score: 253.44
Iteration 3, Best Score: 257.593
Iteration 4, Best Score: 260.57900000000006
Iteration 5, Best Score: 260.57900000000006
Iteration 6, Best Score: 260.57900000000006
Iteration 7, Best Score: 260.57900000000006
Iteration 8, Best Score: 260.57900000000006
Iteration 9, Best Score: 260.57900000000006
Iteration 10, Best Score: 260.57900000000006
Iteration 11, Best Score: 260.57900000000006
Iteration 12, Best Score: 260.57900000000006
Iteration 13, Best Score: 260.57900000000006
Iteration 14, Best Score: 260.57900000000006
Iteration 15, Best Score: 260.57900000000006
Iteration 16, Best Score: 260.57900000000006
Iteration 17, Best Score: 260.57900000000006
Iteration 18, Best Score: 260.57900000000006
Iteration 19, Best Score: 260.57900000000006
Iteration 20, Best Score: 260.57900000000006
Iteration 21, Best Score: 260.57900000000006
Iteration 22, Best Score: 260.57900000000006
Iteration 23, Best Score: 260.57900000000006
Iteratio

## Apparatus
Selected Athletes:
['frederick nathaniel richard' 'paul juda' 'khoi young' 'brody malone']
Total Score:
253.19598690476192

In [15]:
df=df2.copy()

In [16]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['name'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop([ 'name'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['name'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['name']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score


# Set parameters
num_athletes_to_select = 4
tabu_size = 300
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 246.87808412698413
Iteration 2, Best Score: 251.3150674603175
Iteration 3, Best Score: 252.58625079365078
Iteration 4, Best Score: 253.19598690476192
Iteration 5, Best Score: 253.19598690476192
Iteration 6, Best Score: 253.19598690476192
Iteration 7, Best Score: 253.19598690476192
Iteration 8, Best Score: 253.19598690476192
Iteration 9, Best Score: 253.19598690476192
Iteration 10, Best Score: 253.19598690476192
Iteration 11, Best Score: 253.19598690476192
Iteration 12, Best Score: 253.19598690476192
Iteration 13, Best Score: 253.19598690476192
Iteration 14, Best Score: 253.19598690476192
Iteration 15, Best Score: 253.19598690476192
Iteration 16, Best Score: 253.19598690476192
Iteration 17, Best Score: 253.19598690476192
Iteration 18, Best Score: 253.19598690476192
Iteration 19, Best Score: 253.19598690476192
Iteration 20, Best Score: 253.19598690476192
Iteration 21, Best Score: 253.19598690476192
Iteration 22, Best Score: 253.19598690476192
Iteration 23, Best S

# MILP solver 


## IAA

In [66]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/1957350299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [69]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, value, LpBinary
import pandas as pd

# Convert numeric columns to float
numeric_cols = df.columns.difference(['Athlete'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Reset the index to get integer indices
df.reset_index(drop=True, inplace=True)

# Create the LP problem
prob = LpProblem("Gymnast_Selection", LpMaximize)

# Create binary variables for each gymnast-apparatus combination
gymnast_vars = LpVariable.dicts("Compete", ((i, j) for i in range(len(df)) for j in numeric_cols), 0, 1, LpBinary)


# Constraints: Each gymnast competes on up to 6 apparatus
for i in range(len(df)):
    prob += lpSum(gymnast_vars[i, j] for j in numeric_cols) <= 6, f"Gymnast_{i}_Compete_Up_To_4_Apparatus"

# Constraints: Each apparatus is competed on exactly 3 times
for j in numeric_cols:
    prob += lpSum(gymnast_vars[i, j] for i in range(len(df))) == 3, f"{j}_Competed_3_Times"

# Constraint: Select exactly 4 gymnasts
prob += lpSum(gymnast_vars[i, j] for i in range(len(df)) for j in numeric_cols) <= 4, "Select_4_Gymnasts"

# Objective function: maximize the sum of the top 3 scores on each apparatus
prob += lpSum(float(df.loc[i, j]) * gymnast_vars[i, j] for i in range(len(df)) for j in numeric_cols), "Total_Score"


# Solve the problem
prob.solve()

# Display the results
print("Status:", prob.status)
print("Objective Value (Sum of Top 3 Scores):", value(prob.objective))

# Display the selected gymnasts and apparatus
selected_gymnasts = [i for i in range(len(df)) if any(gymnast_vars[i, j].value() == 1 for j in numeric_cols)]

print("\nSelected Gymnasts:")
for i in selected_gymnasts:
    print(df.loc[i, 'Athlete'])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/c5537e185fc045c5b1eadda4724989ef-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/c5537e185fc045c5b1eadda4724989ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 70 COLUMNS
At line 2105 RHS
At line 2171 BOUNDS
At line 2520 ENDATA
Problem MODEL has 65 rows, 348 columns and 1044 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: -1
Objective Value (Sum of Top 3 Scores): 46.51700000000002

Selected Gymnasts:
asher hong
curran phillips
fred richard

## Apparatus

In [196]:
df=df2.copy()

In [197]:
df.head()

,name,fx,hb,pb,ph,sr,vt
0,aidan cuy,0.000000,0.000000,12.950000,0.0000,12.300000,13.800000
1,alex diab,13.225000,0.000000,0.000000,0.0000,14.144167,0.000000
2,asher hong,13.845625,12.072875,14.579444,13.1240,13.999778,14.393643
3,blake sun,0.000000,0.000000,13.673200,13.2232,0.000000,0.000000
4,brandon dang,0.000000,0.000000,0.000000,13.5500,0.000000,0.000000


In [198]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, value, LpBinary
import pandas as pd

# Convert numeric columns to float
numeric_cols = df.columns.difference(['name'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Reset the index to get integer indices
df.reset_index(drop=True, inplace=True)

# Create the LP problem
prob = LpProblem("Gymnast_Selection", LpMaximize)

# Create binary variables for each gymnast-apparatus combination
gymnast_vars = LpVariable.dicts("Compete", ((i, j) for i in range(len(df)) for j in numeric_cols), 0, 1, LpBinary)

# Objective function: maximize the sum of the top 3 scores on each apparatus
prob += lpSum(float(df.loc[i, j]) * gymnast_vars[i, j] for i in range(len(df)) for j in numeric_cols), "Total_Score"

# Constraints: Each gymnast competes on up to 4 apparatus
for i in range(len(df)):
    prob += lpSum(gymnast_vars[i, j] for j in numeric_cols) <= 6, f"Gymnast_{i}_Compete_Up_To_6_Apparatus"

# Constraints: Each apparatus is competed on exactly 3 times
for j in numeric_cols:
    prob += lpSum(gymnast_vars[i, j] for i in range(len(df))) == 3, f"{j}_Competed_3_Times"

# Constraint: Select exactly 4 gymnasts
prob += lpSum(gymnast_vars[i, j] for i in range(len(df)) for j in numeric_cols) == 4, "Select_4_Gymnasts"



# Solve the problem
prob.solve()

# Display the results
print("Status:", prob.status)
print("Objective Value (Sum of Top 3 Scores):", value(prob.objective))

# Display the selected gymnasts and apparatus
selected_gymnasts = [i for i in range(len(df)) if any(gymnast_vars[i, j].value() == 1 for j in numeric_cols)]

print("\nSelected Gymnasts:")
for i in selected_gymnasts:
    print(df.loc[i, 'name'])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/3a1b910c36c84b869fbfddb9cc2a0b6d-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/3a1b910c36c84b869fbfddb9cc2a0b6d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 70 COLUMNS
At line 2067 RHS
At line 2133 BOUNDS
At line 2482 ENDATA
Problem MODEL has 65 rows, 348 columns and 1044 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: -1
Objective Value (Sum of Top 3 Scores): 52.843010317460326

Selected Gymnasts:
brody malone
connor mccool
frederick n

# brute-force search

## Apparatus
Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365

## IAA

Selected Athletes: ['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer'] Maximum Score: 173.974


---

## Apparatus


In [130]:
df=df2.copy()

In [137]:
df.head()

,name,fx,hb,pb,ph,sr,vt
0,aidan cuy,0.000000,0.000000,12.950000,0.0000,12.300000,13.800000
1,alex diab,13.225000,0.000000,0.000000,0.0000,14.144167,0.000000
2,asher hong,13.845625,12.072875,14.579444,13.1240,13.999778,14.393643
3,blake sun,0.000000,0.000000,13.673200,13.2232,0.000000,0.000000
4,brandon dang,0.000000,0.000000,0.000000,13.5500,0.000000,0.000000


Over 3 hours to run
100%|█████████████████████████████████| 424270/424270 [3:13:04<00:00, 36.62it/s]

Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365


In [139]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['fx', 'ph', 'sr', 'vt', 'pb', 'hb']

# Generate all possible combinations of athletes
athlete_combinations = list(itertools.combinations(df['name'], num_athletes))

# store the maximum score and the selected athletes
max_score = 0
selected_athletes = []

# Create a tqdm progress bar
progress_bar = tqdm(total=len(athlete_combinations))

# Iterate combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for current athlete combination
    apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))

    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate sum of  top three scores for  current apparatus combination
        score = df[df['name'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    # Update progress bar
    progress_bar.update(1)

# Close progress bar
progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)

100%|█████████████████████████████████| 424270/424270 [3:13:04<00:00, 36.62it/s]

Selected Athletes:
['aidan cuy', 'asher hong', 'paul juda', 'yul moldauer']
Maximum Score:
169.7672115079365


## IAA

Selected Athletes:
['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']
Maximum Score:
173.974

In [76]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/1957350299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [141]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Generate combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# Initialize variables to store  maximum score and  selected athletes
max_score = 0
selected_athletes = []

progress_bar = tqdm(total=len(athlete_combinations))

# Iterate through  combination of athletes
for combination in athlete_combinations:
    # Generate combinations of apparatuses for  current athlete combination
    apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))

    # Iterate through combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate sum of top three scores for current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    progress_bar.update(1)

progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)

100%|██████████████████████████████| 1581580/1581580 [11:59:24<00:00, 36.64it/s]

Selected Athletes:
['vitaliy guimaraes', 'khoi young', 'asher hong', 'yul moldauer']
Maximum Score:
173.974


---

# Simulated Annealing
- Simulated annealing involves randomness in accepting worse solutions to explore the search space effectively. After adding Seed the results are terrible


Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Maximum Score: 174.61700000000002



In [74]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/1957350299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [75]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    total_score = df[df['Athlete'].isin(selected_athletes)].iloc[:, 2:8].values
    total_score.sort(axis=1, kind='quicksort')
    return total_score[:, -3:].sum()

def simulated_annealing(df, num_athletes=4, iterations=1000, initial_temperature=1.0, cooling_rate=0.95):
    np.random.seed(88)  # Set the seed for reproducibility

    current_solution = np.random.choice(df['Athlete'], size=num_athletes, replace=False).tolist()
    current_score = calculate_score(current_solution, df)

    best_solution = current_solution.copy()
    best_score = current_score

    temperature = initial_temperature

    for iteration in range(iterations):
        neighbor_solution = current_solution.copy()
        idx1, idx2 = np.random.choice(num_athletes, size=2, replace=False)
        neighbor_solution[idx1], neighbor_solution[idx2] = neighbor_solution[idx2], neighbor_solution[idx1]

        neighbor_score = calculate_score(neighbor_solution, df)

        if neighbor_score > current_score or np.random.rand() < np.exp((neighbor_score - current_score) / temperature):
            current_solution = neighbor_solution
            current_score = neighbor_score

        if current_score > best_score:
            best_solution = current_solution.copy()
            best_score = current_score

        temperature *= cooling_rate

    return best_solution, best_score


# Run simulated annealing
best_athletes, best_score = simulated_annealing(df)

# Print the results
print('Selected Athletes:', best_athletes)
print('Maximum Score:', best_score)


Selected Athletes: ['fuzzy benas', 'yul moldauer', 'asher hong', 'fred richard']
Maximum Score: 174.61700000000002


# Harmony Search algorithm = results for 6 and not 4 athletes

              Athlete     FX      PH     SR     VT      PB     HB
715    patrick hoopes   0.00  14.992   0.00   0.00   0.000   0.00
708   curran phillips   0.00   0.000   0.00   0.00  15.997  13.70
427   ian lasic-ellis  13.65  13.500  13.75  13.55  13.300  13.30
711   spencer goodell  13.45   0.000   0.00   0.00  13.750   0.00
442      noah newfeld  12.20  13.050  12.10  13.50  13.200  12.15
444  garrett braunton  12.85  12.900  13.05  12.20  13.150  11.65

In [70]:
iaa24 = pd.read_csv('../../Combine_Data/men/iaa2024.csv') 
usa = iaa24[iaa24['Nation'] == 'USA']

usa.fillna(0, inplace=True)

# Group by 'Athlete' and get the maximum value for each athlete
df_max = usa.groupby('Athlete').max()

# Reset the index to make 'Athlete' a column again
df_max.reset_index(inplace=True)
df=df_max.copy()
df=df.drop(columns=['Rank', 'AA', 'round', 'Nation', 'year'])

df.head()

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_95635/1957350299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa.fillna(0, inplace=True)


,Athlete,FX,PH,SR,VT,PB,HB
0,alex diab,13.550,0.000,14.698,13.552,0.000,0.00
1,anthony koppie,11.900,4.650,12.550,13.950,11.750,11.40
2,asher hong,14.029,12.949,14.566,16.680,14.700,13.65
3,blake sun,0.000,13.500,0.000,0.000,15.044,0.00
4,brandon dang,0.000,14.466,0.000,0.000,0.000,0.00


In [71]:


import pandas as pd
import numpy as np

def fitness(solution, df):
    selected_athletes = df.iloc[solution]
    
    # Check if each apparatus is competed exactly three times
    apparatus_counts = selected_athletes.groupby('Athlete').size()
    if not all(count == 3 for count in apparatus_counts):
        return np.inf
    
    # Check if the selected athletes are unique for each apparatus
    if len(set(solution)) < len(solution):
        return np.inf
    
    scores = selected_athletes[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].values
    top3_sum = np.sum(np.partition(scores, -3)[:, -3:], axis=1)
    
    return -np.sum(top3_sum)  # Negative because Harmony Search seeks to minimize the objective function

def harmony_search(df, num_iterations, harmony_memory_size):
    num_athletes = len(df)
    num_apparatus = 6  # Assuming 6 apparatus (FX, PH, SR, VT, PB, HB)
    hm_size = harmony_memory_size

    # Initialize harmony memory with random solutions
    harmony_memory = np.random.randint(0, num_athletes, size=(hm_size, num_apparatus))

    for iteration in range(num_iterations):
        # Create new harmony solutions
        new_solution = np.random.randint(0, num_athletes, size=num_apparatus)

        # Update the harmony memory with the new solution
        worst_index = np.argmin([fitness(harmony, df) for harmony in harmony_memory])
        
        # Check if the new solution is valid before updating the memory
        if fitness(new_solution, df) < np.inf:
            harmony_memory[worst_index] = new_solution

    # Find the best solution in the harmony memory
    best_solution = harmony_memory[np.argmin([fitness(harmony, df) for harmony in harmony_memory])]

    return df.iloc[best_solution]

# Assuming 'your_dataframe.csv' is the path to your CSV file
# df = pd.read_csv('your_dataframe.csv')

# Set the number of iterations and harmony memory size
num_iterations = 100
harmony_memory_size = 10

# Run the Harmony Search algorithm
selected_athletes = harmony_search(df, num_iterations, harmony_memory_size)

# Print the selected athletes
print(selected_athletes[['Athlete', 'FX', 'PH', 'SR', 'VT', 'PB', 'HB']])


           Athlete     FX     PH     SR      VT      PB     HB
3        blake sun   0.00  13.50   0.00   0.000  15.044   0.00
54  vahe petrosyan   0.00   0.00   0.00  13.350   0.000   0.00
15      evan davis  13.35  11.35  13.20  14.100  13.250  12.55
35    matt cormier  11.05  12.10  13.50  14.200  13.450  12.95
39   mike fletcher  12.60  13.35  13.15  14.450  13.550  12.40
10     colt walker  12.45  12.75  13.25  14.897  15.055  13.10
